In [13]:
from rdflib import Graph,Literal,RDF,URIRef, RDFS
from rdflib.namespace import XSD, FOAF
from rdflib import Namespace
import pandas as pd
from sodapy import Socrata

In [29]:
from rdflib.namespace import DCTERMS, DC

In [30]:
# pip install sodapy

In [31]:
g = Graph()
g.parse('NSOD_Ont.owl')


<Graph identifier=N821d2db0d97d434493171f174cfc6bd0 (<class 'rdflib.graph.Graph'>)>

In [32]:
g1 = Graph()
g1.parse('doid_2021.owl')

<Graph identifier=N2a80d5f514874a9b8f6ebf60b2383b17 (<class 'rdflib.graph.Graph'>)>

In [35]:
cube = Namespace("http://purl.org/linked-data/cube#")
sdmxdimension = Namespace("http://purl.org/linked-data/sdmx/2009/dimension#")
geo = Namespace ("https://www.geonames.org/")
sdmx = Namespace ("http://purl.org/linked-data/sdmx#")
freq = Namespace ("http://purl.org/cld/freq/")
doid = Namespace ("http://purl.obolibrary.org/obo/DOID#")
eg = Namespace('http://www.semanticweb.org/erajabi/ontologies/nsod#')
dcat = Namespace('http://www.w3.org/ns/dcat#')
dcterms = Namespace('http://purl.org/dc/terms/')
doid = Namespace('http://purl.obolibrary.org/obo/')
void = Namespace('http://rdfs.org/ns/void#')
foaf = Namespace('http://xmlns.com/foaf/0.1/')

g.bind('qb', cube)
g.bind('eg', eg)
g.bind('dimension', sdmxdimension)
g.bind('dcat', dcat)
g.bind('dcterms', dcterms)
g.bind('sdmx', sdmx)
g.bind('doid', doid)
g.namespace_manager.bind('doid', URIRef('http://purl.obolibrary.org/obo/'))

list_of_api={"sr7h-uyxt",    #Tetanus
             "naay-xy8i",    #AIDS
             "5kjg-qidk",    #Salmonellosis
             "r9i7-qnuh",    #HepatitisA
             "w7bm-jav4",    #HepatitisC
             "s4zv-fef7",    #Shigellosis
             "dxhy-v63n",    #Giardiasis
             "sps3-eq7e",    #Chlamydia
             "fbx9-xzkq",    #Legionellosis
             "enig-mjwt",    #Cyclosporiasis
             "5itt-58m2",    #Pertussis
             "4p4c-dv5x",    #Malaria
             "tsr3-8hh2",    #HIV
             "qsy8-u73r",    #Gonorrhea
             "nn2f-jgv4",    #Campylobacteriosis
             "4vij-qbup",    #Botulism
             "5mqu-g4g2",    #Rubella
             "48h6-dubm",    #Tuberculosis
             "e2eh-63yz",    #Mumps
             "p42z-mdp9",    #Typhoid
             "9ckr-t936",    #Cryptosporidiosis
            }
from rdflib.namespace import XSD

client = Socrata("data.novascotia.ca", None)
    
g.add((eg.sliceByYear,RDF.type,cube.SliceKey))
g.add((eg.sliceByYear,RDFS.label,Literal("slice by year", datatype=XSD.string)))
g.add((eg.sliceByYear,eg.componentProperty,eg.refPeriod))


g.add((eg.slice2015,RDF.type,cube.Slice))
g.add((eg.slice2015,cube.sliceStructure,eg.sliceByYear))
g.add((eg.slice2015,eg.refPeriod,Literal(2015, datatype=XSD.integer)))

In [36]:
regioniri_NS=geo+"6091530"
region_NS=URIRef(regioniri_NS)
g.add((region_NS,RDFS.label,Literal("Nova Scotia",datatype=XSD.string)))


for i in list_of_api:
    results = client.get(i)
    results1 = client.get_metadata(i)
    results_df = pd.DataFrame.from_records(results)
    name=results1["name"].replace(' ','')
    name=Literal(name, datatype=XSD.string)
    comment=Literal(results1["description"],datatype=XSD.string)
    license=URIRef("https://novascotia.ca/opendata/licence.asp")
    Category=results1["category"]
    Category=Category.replace(" ","")
    Category=eg+Category
    theme=URIRef(Category)
    urlhealthandwellness="https://data.novascotia.ca/Health-and-Wellness/"
    urldataset=urlhealthandwellness+name+'/'+results1["id"]
    isDefinedBy=Literal(urldataset)
    Dept=results1["metadata"]["custom_fields"]["Detailed Metadata"]["Department"]
    Dept=Dept.replace(" ","")
    Dept=eg+Dept
    department=URIRef(Dept)
    DSD=URIRef("http://www.semanticweb.org/erajabi/ontologies/nsod#dsd-diseasedatasets")
    lang=results1["metadata"]["custom_fields"]["Detailed Metadata"]["Language"]
    if len(lang)== 3:
        lang="http://www.lexvo.org/page/iso639-3/"+lang
    lang=URIRef(lang)
    frequency=results1["metadata"]["custom_fields"]["Detailed Metadata"]["Frequency"]
    if frequency=="Annual" or frequency=="Annually"or frequency=="annual"or frequency=="annually"or frequency=="yearly"or frequency=="Yearly":
        frequency=freq.annual
    frequency=URIRef(frequency)
    Location=results1["metadata"]["custom_fields"]["Detailed Metadata"]["Geographic Region Name"]

    dataset=eg+'dataset-'+name
    dataset= URIRef(dataset)
    g.add((dataset,RDF.type,cube.DataSet))
    g.add((dataset,RDFS.label,name))
    g.add((dataset,RDFS.comment,comment))
    g.add((dataset,DCTERMS.license,license))
#     g.add((dataset,DCAT.theme,theme))
    g.add((dataset,RDF.type,void.Dataset))

    for i in results1["tags"]:
        tag=i
        tag=tag.replace(" ","")
        new=eg+tag
        keyword=URIRef(new)
#         g.add((dataset,DCAT.keyword,keyword))
    g.add((dataset,foaf.homepage,isDefinedBy))
    g.add((dataset,eg.department,department))
    g.add((dataset,cube.structure,DSD))
    g.add((dataset,DCTERMS.language,lang))
    g.add((dataset,DCTERMS.publisher,'Nova Scotia Open Data Portal'))
    g.add((dataset,DCTERMS.accrualPeriodicity,frequency))
    if(Location == 'Nova Scotia'):
        g.add((dataset,DCTERMS.spatial,region_NS))
        region_selected = region_NS

    for i in range(1,len(results_df)):
        x=str(i)
        obsiri=eg+"obs-"+name+"-"+x
        obs=URIRef(obsiri)
        g.add((obs,cube.dataSet,dataset))
        g.add((obs,RDF.type,cube.Observation))
        g.add((obs,sdmxdimension.refArea,region_selected))

        yr_original=str(results_df["year"][i])
#         yr='https://dbpedia.org/page/'+yr_original
#         year=URIRef(yr)
        g.add((obs,sdmxdimension.refPeriod,Literal(results_df["year"][i], datatype=XSD.integer)))
        regioniri=geo+"6091530"
        region=URIRef(regioniri)
        T=sdmx.sex+"-T"
        T = URIRef(T)
        g.add((obs,sdmx.sex,T))
        numberofcases=Literal(results_df["number_of_cases"][i],datatype=XSD.integer)
        g.add((obs,eg.numberofcases,numberofcases))
        rate_per_100_000_population=Literal(results_df["rate_per_100_000_population"][i],datatype=XSD.float)
        g.add((obs,eg.rateper100kpopulation,numberofcases))
        disease=results_df["disease"][i]
        disease=disease.lower()
        if (disease=="human immunodeficiency virus"):
            disease="human immunodeficiency virus infectious disease"
        elif (disease=="typhoid"):
            disease="typhoid fever"
        elif (disease=="acquired immune deficiency syndrome"):
            disease="acquired immunodeficiency syndrome"
        elif (disease=="hepatitis c"):
            disease="hepatitis C"
        elif (disease=="hepatitis a"):
            disease="hepatitis A"
        diseaselabel=Literal(disease, datatype=XSD.string)
        qres = g1.query("""SELECT ?o ?superclass ?superClassLabel WHERE { ?o rdfs:subClassOf ?superclass. ?superclass rdfs:label ?superClassLabel.
                ?o rdfs:label ?label. }""",
                initNs ={"rdf": RDF,"rdfs": RDFS},
                initBindings={"label": diseaselabel })

        if(len(qres)>0):
            g.add((obs,RDFS.label,diseaselabel))
            for row in qres:
#                 g.add((obs,eg.hasdisease,URIRef(row.superclass)))
                g.add((obs,eg.hasdisease,URIRef(row.o)))
                g.add((URIRef(row.o),RDF.type,eg.Disease))
#                 g.add((URIRef(row.o),RDF.type,URIRef(row.superclass)))
                g.add((URIRef(row.o),RDFS.label,diseaselabel))
                g.add((URIRef(row.superclass),RDF.type,eg.Disease))
                g.add((URIRef(row.superclass),RDFS.label,row.superClassLabel))
                g.add((URIRef(row.o),doid.is_a,URIRef(row.superclass)))

        if(yr_original=='2015'):
                g.add((eg.slice2015,cube.observation,obs))  
        
#     break
g.serialize('NSOD_KG.rdf',format='xml')      
print('KG was created')

KG was created


In [488]:
diseaselabel = Literal("hepatitis C")
qres = g1.query("""SELECT ?o ?s1 ?superClassLabel WHERE { ?o rdfs:subClassOf ?s1. ?s1 rdfs:label ?superClassLabel.
                ?o rdfs:label ?label. }""",
                initNs ={"rdf": RDF,"rdfs": RDFS},
                initBindings={"label": diseaselabel })

# qres = g1.query("""SELECT ?s ?p ?o WHERE { ?s ?p ?o. ?o <http://www.w3.org/2000/01/rdf-schema#label> "tuberculosis"}""")

if(len(qres)>0):
    for row in qres:
        print(f"{row.o} ({row.s1}) {row.superClassLabel}")


